In [13]:
import os
import cv2
import copy
import keras
import pickle
import tensorflow as tf

In [14]:
INT_S1_DIR = "_intermediate/stage1"
INT_S2_DIR = "_intermediate/stage2"

PROB_THRESHOLD = .5

In [15]:
os.makedirs(INT_S2_DIR, exist_ok=True)

In [16]:
def get_latest_checkpoint(dir):
    if not os.path.isdir(dir):
        return None
    ckpts = [file for file in os.listdir(dir) if file.endswith("keras")]
    ckpts.sort()
    return os.path.join(dir, ckpts[-1])

In [17]:
with open(os.path.join(INT_S1_DIR, "total_data.pkl"), 'rb') as file:
    stage1_results = pickle.load(file)

In [18]:
stage1_results

{'SAM + YOLOv8': {932: {'healthy': False,
   'masks': [{'segmentation': array([[False, False, False, ..., False, False, False],
            [False, False, False, ..., False, False, False],
            [False, False, False, ..., False, False, False],
            ...,
            [False, False, False, ..., False, False, False],
            [False, False, False, ..., False, False, False],
            [False, False, False, ..., False, False, False]]),
     'area': 1179930,
     'bbox': [346, 187, 1604, 1011],
     'predicted_iou': 1.0618212223052979,
     'point_coords': [[736.0, 618.515625]],
     'stability_score': 0.9912988543510437,
     'crop_box': [0, 0, 2048, 1365],
     'patch': array([[[120, 124,  82],
             [120, 124,  82],
             [120, 124,  82],
             ...,
             [ 35, 130,  86],
             [ 36, 131,  87],
             [ 34, 129,  85]],
     
            [[119, 123,  81],
             [119, 123,  81],
             [119, 123,  81],
             ...,


In [19]:
stage2_names = ["InceptionV3", "VGG19", "MobileNetV3Large"]

In [20]:
# Load the latest checkpoints
stage2_dict = {
    name: keras.models.load_model(get_latest_checkpoint(f"../disease_detection/checkpoints/{name}_rgb")) 
    for name in stage2_names 
    if get_latest_checkpoint(f"../disease_detection/checkpoints/{name}_rgb") is not None
}

2024-11-16 11:10:23.453953: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 37412 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:01:00.0, compute capability: 8.0


In [21]:
# Load best performing model by type
stage2_dict = {
    name: keras.models.load_model(f"../disease_detection/out/best{name}_rgb.keras")
    for name in stage2_names 
}

In [22]:
stage2_results = {}
for stage2_name, stage2_model in stage2_dict.items():
    for stage1_name, stage1_result in stage1_results.items():
        stage_name = f'({stage1_name}) + ({stage2_name})'
        print(f'Evaluating {stage_name}')
        stage2_results[stage_name] = copy.deepcopy(stage1_result)
        for index, data in stage2_results[stage_name].items():
            healthy = data['healthy']
            masks = data['masks']
            data['predictions'] = []
            for mask in masks:
                patch = mask['patch']
                inputs = cv2.resize(patch, (224, 224))
                inputs = tf.constant(inputs)
                inputs = tf.expand_dims(inputs, 0)
                prob = stage2_model(inputs).numpy().squeeze()[1]
                data['predictions'].append(prob)
                mask['prob_diseased'] = prob
                mask['predicted_sick'] = int(prob > PROB_THRESHOLD)

Evaluating (SAM + YOLOv8) + (InceptionV3)


2024-11-16 11:10:37.300213: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
W0000 00:00:1731751837.371853 2232016 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1731751837.404074 2232016 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1731751837.409921 2232016 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1731751837.410286 2232016 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1731751837.434715 2232016 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1731751837.435090 2232016 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1731751837.435423 2232016 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1731751837.435761 2232016 gpu_t

Evaluating (SAM + ResNet) + (InceptionV3)
Evaluating (SAM + YOLOv8) + (VGG19)


W0000 00:00:1731751850.662324 2232016 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1731751850.662761 2232016 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1731751850.663191 2232016 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1731751850.663525 2232016 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1731751850.663843 2232016 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1731751850.664190 2232016 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1731751850.664545 2232016 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1731751850.664896 2232016 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1731751850.665262 2232016 gp

Evaluating (SAM + ResNet) + (VGG19)
Evaluating (SAM + YOLOv8) + (MobileNetV3Large)


W0000 00:00:1731751853.112589 2232016 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1731751853.112924 2232016 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1731751853.113221 2232016 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1731751853.113503 2232016 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1731751853.113791 2232016 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1731751853.114073 2232016 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1731751853.114344 2232016 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1731751853.114613 2232016 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1731751853.114954 2232016 gp

Evaluating (SAM + ResNet) + (MobileNetV3Large)


In [23]:
stage2_results

{'(SAM + YOLOv8) + (InceptionV3)': {932: {'healthy': False,
   'masks': [{'segmentation': array([[False, False, False, ..., False, False, False],
            [False, False, False, ..., False, False, False],
            [False, False, False, ..., False, False, False],
            ...,
            [False, False, False, ..., False, False, False],
            [False, False, False, ..., False, False, False],
            [False, False, False, ..., False, False, False]]),
     'area': 1179930,
     'bbox': [346, 187, 1604, 1011],
     'predicted_iou': 1.0618212223052979,
     'point_coords': [[736.0, 618.515625]],
     'stability_score': 0.9912988543510437,
     'crop_box': [0, 0, 2048, 1365],
     'patch': array([[[120, 124,  82],
             [120, 124,  82],
             [120, 124,  82],
             ...,
             [ 35, 130,  86],
             [ 36, 131,  87],
             [ 34, 129,  85]],
     
            [[119, 123,  81],
             [119, 123,  81],
             [119, 123,  81],


In [24]:
for name, data in stage2_results.items():
    os.makedirs(os.path.join(INT_S2_DIR, name), exist_ok=True)
    with open(os.path.join(INT_S2_DIR, name, "data.pkl"), 'wb+') as file:
        pickle.dump(data, file, protocol=pickle.HIGHEST_PROTOCOL)

In [27]:
with open(os.path.join(INT_S2_DIR, "total.pkl"), "wb+") as file:
    pickle.dump(stage2_results, file, protocol=pickle.HIGHEST_PROTOCOL)

In [26]:
print(stage2_results.keys())

first_key1 = next(iter(stage2_results.keys()))
print(stage2_results[first_key1].keys())

first_key2 = next(iter(stage2_results[first_key1].keys()))
print(stage2_results[first_key1][first_key2]['masks'][0].keys())

dict_keys(['(SAM + YOLOv8) + (InceptionV3)', '(SAM + ResNet) + (InceptionV3)', '(SAM + YOLOv8) + (VGG19)', '(SAM + ResNet) + (VGG19)', '(SAM + YOLOv8) + (MobileNetV3Large)', '(SAM + ResNet) + (MobileNetV3Large)'])
dict_keys([932, 290])
dict_keys(['segmentation', 'area', 'bbox', 'predicted_iou', 'point_coords', 'stability_score', 'crop_box', 'patch', 'leaf_probability', 'prob_diseased', 'predicted_sick'])
